In [8]:
import datetime
import os
import pprint
import tempfile
from typing import Dict, Text

import numpy as np
import pandas as pd
import seaborn as sns
import surprise
import tensorflow as tf
import tensorflow_recommenders as tfrs

import tensorflow_datasets as tfds

In [9]:
df_raw = pd.read_csv("../lovevery_mle_homework_content_engagement.csv")
df_raw

,user_id,content_id,event_timestamp,event_type
0,1438eb1414,f260f3f3ce,1.642347e+09,viewed
1,5380d76552,f260f3f3ce,1.642538e+09,viewed
2,f600b96610,f260f3f3ce,1.643294e+09,engaged
3,395ee54b4e,bce874342a,1.642697e+09,viewed
4,dcbb8d2a04,f260f3f3ce,1.641336e+09,viewed
...,...,...,...,...
484072,3638877804,a842a97d1a,1.644180e+09,viewed
484073,e1903bb0ba,5c57c25574,1.642185e+09,viewed
484074,f5fa01d9b5,58bb410bc8,1.643659e+09,engaged
484075,0090363528,5c57c25574,1.640085e+09,viewed


# Build Datasets

## Make Engaged DataFrame
* every record in raw data is either a view or engagement.
* The rate a user engages with content they see is an implicit rating
* as seen in the pivot table user_id vs content_id, not all content & user_id combinations have been presented. This is what the model trys to predict
* all users in dataset have engaged with at least one piece of content

In [10]:
# create a column for the number of times content was viewed. Returns 197,562 records
df_viewed = (
    df_raw.groupby(["user_id", "content_id"])["event_type"]
    .apply(lambda x: (x == "viewed").sum())
    .reset_index(name="viewed_count")
)

In [11]:
# create a column for the number of times content was engaged. Returns 197,562 records
df_engaged = (
    df_raw.groupby(["user_id", "content_id"])["event_type"]
    .apply(lambda x: (x == "engaged").sum())
    .reset_index(name="engaged_count")
)

In [12]:
df = pd.merge(df_viewed, df_engaged, on=["user_id", "content_id"], how="inner")

In [13]:
# can have instances where engaged count is higher than viewed count.
# Must add the two columns to get a total number of times a user has seen content
df["shown_count"] = df["engaged_count"] + df["viewed_count"]

In [14]:
# engaged pct is a proxy for some type of rating.
df["engaged_pct"] = df["engaged_count"] / df["shown_count"]

In [15]:
df.sort_values(by="shown_count")

,user_id,content_id,viewed_count,engaged_count,shown_count,engaged_pct
0,000a544834,34caa29b86,1,0,1,0.000000
98451,82216a2c7d,bd1201fb2a,0,1,1,1.000000
98444,82216a2c7d,86091a3dd6,0,1,1,1.000000
98440,82216a2c7d,639ae113f9,0,1,1,1.000000
98438,82216a2c7d,5149306338,1,0,1,0.000000
...,...,...,...,...,...,...
70255,5e0fb13f54,8b53482986,20,9,29,0.310345
163060,d39bca4a9c,0e5f24c27b,35,0,35,0.000000
163065,d39bca4a9c,99c6cf7932,19,32,51,0.627451
136619,b104fe8d85,563c1fdf8f,24,33,57,0.578947


In [16]:
# ratings -> engageed_pct
df_ratings = df[["user_id", "content_id", "engaged_pct"]]
len(df_ratings["content_id"].unique())

583

In [17]:
df_ratings.head()

,user_id,content_id,engaged_pct
0,000a544834,34caa29b86,0.0
1,000a544834,3d0e786812,0.0
2,000a544834,5c7ee2dd80,0.5
3,000a544834,6f7ca40e85,0.0
4,000a544834,745115bd62,0.0


## Make Content_id Dataframe for Model
includes ids for all content

In [18]:
df_content = pd.DataFrame(df_raw["content_id"].unique())
df_content = df_content.rename(columns={0: "content_id"})
df_content.head()

,content_id
0,f260f3f3ce
1,bce874342a
2,bbf2753b99
3,a9b6c93c3a
4,0c93cc1338


In [19]:
df_content["content_id"]

0      f260f3f3ce
1      bce874342a
2      bbf2753b99
3      a9b6c93c3a
4      0c93cc1338
          ...    
578    2ca0fce09f
579    5c57c25574
580    58bb410bc8
581    a842a97d1a
582    2ce01b7f37
Name: content_id, Length: 583, dtype: object

# Modeling
Implement both retrieval and ranking models

Retrieval - two tower - https://blog.tensorflow.org/2020/09/introducing-tensorflow-recommenders.html
takes a user id and returns a subset of good candidates

ranking - https://www.tensorflow.org/recommenders/examples/basic_ranking
takes ad user id and a subset of good candidates and returns the best candidate

optimizing for a single objective (multi object is avaailable - ratings and clicks)

In [ ]:
mlflow.tensorflow.autolog()

USER_ID = "82216a2c7d"
CONTENT_ID = "49a9f915b5"

## Retrieval

### Load data into TensorFlow

In [21]:
ratings_raw = tf.data.Dataset.from_tensor_slices(
    dict(df_ratings[["user_id", "content_id", "engaged_pct"]])
)
content_raw = tf.data.Dataset.from_tensor_slices(dict(df_content[["content_id"]]))
print(ratings_raw.element_spec)
print(content_raw.element_spec)

{'user_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'content_id': TensorSpec(shape=(), dtype=tf.string, name=None), 'engaged_pct': TensorSpec(shape=(), dtype=tf.float64, name=None)}
{'content_id': TensorSpec(shape=(), dtype=tf.string, name=None)}


In [22]:
ratings_retrieval = ratings_raw.map(
    lambda x: {"user_id": x["user_id"], "content_id": x["content_id"]}
)

content_retrieval = content_raw.map(lambda x: x["content_id"])

print("users_content length: ", len(ratings_retrieval))
print("content length: ", len(content_retrieval))

users_content length:  197562
content length:  583


In [23]:
# Ratings: Shuffle and split into train and test

tf.random.set_seed(42)
shuffled_ratings = ratings_retrieval.shuffle(
    buffer_size=1_000, seed=42, reshuffle_each_iteration=False
)
print("length of shuffled: ", len(shuffled_ratings))

to_take_ratings = int(len(shuffled_ratings) * 0.75)
remaining_ratings = int(len(shuffled_ratings) - to_take_ratings)

print("training set size: ", to_take_ratings)
print("test set size: ", remaining_ratings)

train_retrieval = shuffled_ratings.take(to_take_ratings)
test_retrieval = shuffled_ratings.skip(to_take_ratings).take(remaining_ratings)

length of shuffled:  197562
training set size:  148171
test set size:  49391


In [25]:
# get uniques

content_ids_retrieval = content_retrieval.batch(1_000)
user_ids_retrieval = ratings_retrieval.batch(1_000).map(lambda x: x["user_id"])

unique_content_ids_retrieval = np.unique(np.concatenate(list(content_ids_retrieval)))
unique_user_ids_retrieval = np.unique(np.concatenate(list(user_ids_retrieval)))

print("number of unique titles: ", len(unique_content_ids_retrieval))
print("number of unique users: ", len(unique_user_ids_retrieval))

# look at top 10
unique_content_ids_retrieval[:5]

number of unique titles:  583
number of unique users:  7252


array([b'00733d9430', b'008c372b81', b'01af663e58', b'024775977e',
       b'02538907fb'], dtype=object)

### Define Model

In [26]:
embedding_dimension = 32

In [27]:
user_retrieval_model = tf.keras.Sequential(
    [
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids_retrieval, mask_token=None
        ),
        tf.keras.layers.Embedding(
            len(unique_user_ids_retrieval) + 1, embedding_dimension
        ),
    ]
)

In [30]:
content_retrieval_model = tf.keras.Sequential(
    [
        tf.keras.layers.StringLookup(
            vocabulary=unique_content_ids_retrieval, mask_token=None
        ),
        tf.keras.layers.Embedding(
            len(unique_content_ids_retrieval) + 1, embedding_dimension
        ),
    ]
)

metrics = tfrs.metrics.FactorizedTopK(
    candidates=content_retrieval.batch(128).map(content_retrieval_model)
)
TASK_RETRIEVAL = tfrs.tasks.Retrieval(metrics=metrics)

In [31]:
# defined as the dot product between user and item embeddings. corresponds to matrix factorization.


class RetrievalModel(tfrs.Model):
    def __init__(self, user_model, content_model):
        super().__init__()
        self.content_model: tf.keras.Model = content_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = TASK_RETRIEVAL

    def compute_loss(
        self, features: Dict[Text, tf.Tensor], training=False
    ) -> tf.Tensor:
        # We pick out the user features and pass them into the user model.
        user_embeddings = self.user_model(features["user_id"])

        # And pick out the content features and pass them into the coontent model,
        # getting embeddings back.
        positive_content_embeddings = self.content_model(features["content_id"])

        # The task computes the loss and the metrics.
        return self.task(user_embeddings, positive_content_embeddings)

### Fit and Evaluate

In [32]:
model_retrieval = RetrievalModel(user_retrieval_model, content_retrieval_model)
model_retrieval.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [33]:
cached_train_retrieval = (
    train_retrieval.shuffle(buffer_size=100_000).batch(8192).cache()
)
cached_test_retrieval = test_retrieval.batch(4096).cache()

In [34]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [35]:
model_retrieval.fit(cached_train_retrieval, epochs=3, callbacks=[tensorboard_callback])
# To access TensorBoard - run the command below in terminal
# tensorboard --logdir logs/fit

Epoch 1/3
19/19 [==============================] - 16s 674ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0099 - factorized_top_k/top_5_categorical_accuracy: 0.0754 - factorized_top_k/top_10_categorical_accuracy: 0.1567 - factorized_top_k/top_50_categorical_accuracy: 0.5234 - factorized_top_k/top_100_categorical_accuracy: 0.7310 - loss: 60905.8454 - regularization_loss: 0.0000e+00 - total_loss: 60905.8454
Epoch 2/3
19/19 [==============================] - 11s 591ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0169 - factorized_top_k/top_5_categorical_accuracy: 0.1229 - factorized_top_k/top_10_categorical_accuracy: 0.2415 - factorized_top_k/top_50_categorical_accuracy: 0.6866 - factorized_top_k/top_100_categorical_accuracy: 0.9073 - loss: 54177.5782 - regularization_loss: 0.0000e+00 - total_loss: 54177.5782
Epoch 3/3
19/19 [==============================] - 12s 652ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0178 - factorized_top_k/top_5_categorical_accurac

In [36]:
model_retrieval.evaluate(cached_test_retrieval, return_dict=True)

13/13 [==============================] - 6s 315ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0015 - factorized_top_k/top_5_categorical_accuracy: 0.0117 - factorized_top_k/top_10_categorical_accuracy: 0.0231 - factorized_top_k/top_50_categorical_accuracy: 0.0993 - factorized_top_k/top_100_categorical_accuracy: 0.1865 - loss: 29309.0101 - regularization_loss: 0.0000e+00 - total_loss: 29309.0101


{'factorized_top_k/top_1_categorical_accuracy': 0.0015184952644631267,
 'factorized_top_k/top_5_categorical_accuracy': 0.01172278355807066,
 'factorized_top_k/top_10_categorical_accuracy': 0.02310137450695038,
 'factorized_top_k/top_50_categorical_accuracy': 0.09932983666658401,
 'factorized_top_k/top_100_categorical_accuracy': 0.18653196096420288,
 'loss': 1312.309814453125,
 'regularization_loss': 0,
 'total_loss': 1312.309814453125}

### Get Content Recommendations - Brute Force
The most straightforward way of retrieving top candidates in response to a query is to do it via brute force: compute user-movie scores for all possible movies, sort them, and pick a couple of top recommendations.

As the number of candidate grows, the amount of time needed grows linearly: with 10 million candidates, serving top candidates would take 250 milliseconds. This is clearly too slow for a live service. This is where approximate mechanisms come in. We can use ScaNN in TFRS to accomplish this via the tfrs.layers.factorized_top_k.ScaNN layer.

In [58]:
# Create BruteForce model that takes in raw user_id features, and
brute_force_index = tfrs.layers.factorized_top_k.BruteForce(model_retrieval.user_model)

# recommends content out of the entire content dataset.
brute_force_index.index_from_dataset(
    tf.data.Dataset.zip(
        (
            content_retrieval.batch(100),
            content_retrieval.batch(100).map(model_retrieval.content_model),
        )
    )
)

# Get recommendations for:
_, content_ids_recommended = brute_force_index(tf.constant([USER_ID]))
print(
    f"content_id recommendations for user {USER_ID}: {content_ids_recommended[0, :5]}"
)

content_id recommendations for user 82216a2c7d: [b'49a9f915b5' b'a533f2f57a' b'20b0745a31' b'487d868f96' b'19e9799800']


In [38]:
# This system is for: user -> item
# item -> item. usualy on a product page to recommend similar products. Can get with cosine similarity measure vs dot product for matrix factorization

## Ranking
* pointwise - candidates are considered independently of each other (currently using)
* listwise - candidates pairs are considered, consider the ordering of the entire list
* can use explicit feedback (engage pct) as a ranking

### Load Data into TensorFlow

In [39]:
ratings_raw.element_spec

{'user_id': TensorSpec(shape=(), dtype=tf.string, name=None),
 'content_id': TensorSpec(shape=(), dtype=tf.string, name=None),
 'engaged_pct': TensorSpec(shape=(), dtype=tf.float64, name=None)}

In [40]:
# use user_rating
ratings_ranking = ratings_raw.map(
    lambda x: {
        "user_id": x["user_id"],
        "content_id": x["content_id"],
        "engaged_pct": x["engaged_pct"],
    }
)

In [42]:
# suffle and train/test split
tf.random.set_seed(42)
shuffled_ranking = ratings_ranking.shuffle(
    buffer_size=1_000, seed=42, reshuffle_each_iteration=False
)
print("length of shuffled: ", len(shuffled_ranking))

to_take_ranking = int(len(shuffled_ranking) * 0.75)
remaining_ranking = int(len(shuffled_ranking) - to_take_ranking)

print("training set size: ", to_take_ranking)
print("test set size: ", remaining_ranking)

train_ranking = shuffled_ranking.take(to_take_ranking)
test_ranking = shuffled_ranking.skip(to_take_ranking).take(remaining_ranking)

shuffled_ranking.element_spec

length of shuffled:  197562
training set size:  148171
test set size:  49391


{'user_id': TensorSpec(shape=(), dtype=tf.string, name=None),
 'content_id': TensorSpec(shape=(), dtype=tf.string, name=None),
 'engaged_pct': TensorSpec(shape=(), dtype=tf.float64, name=None)}

In [43]:
# get uniques. all from ratings
content_ids_ranking = ratings_ranking.batch(1_000_000).map(lambda x: x["content_id"])
user_ids_ranking = ratings_ranking.batch(1_000_000).map(lambda x: x["user_id"])

unique_content_ids_ranking = np.unique(np.concatenate(list(content_ids_ranking)))
unique_user_ids_ranking = np.unique(np.concatenate(list(user_ids_ranking)))

### Define Model

In [47]:
class RankingDefinitionModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        embedding_dimension = 32

        # Compute embeddings for users.
        self.user_embeddings = tf.keras.Sequential(
            [
                tf.keras.layers.StringLookup(
                    vocabulary=unique_user_ids_ranking, mask_token=None
                ),
                tf.keras.layers.Embedding(
                    len(unique_user_ids_ranking) + 1, embedding_dimension
                ),
            ]
        )

        # Compute embeddings for content_ids.
        self.content_embeddings = tf.keras.Sequential(
            [
                tf.keras.layers.StringLookup(
                    vocabulary=unique_content_ids_ranking, mask_token=None
                ),
                tf.keras.layers.Embedding(
                    len(unique_content_ids_ranking) + 1, embedding_dimension
                ),
            ]
        )

        # Compute predictions.
        self.ratings = tf.keras.Sequential(
            [
                # Learn multiple dense layers.
                tf.keras.layers.Dense(256, activation="relu"),
                tf.keras.layers.Dense(64, activation="relu"),
                # Make rating predictions in the final layer.
                tf.keras.layers.Dense(1),
            ]
        )

    def call(self, inputs):

        user_id, content_id = inputs

        user_embedding = self.user_embeddings(user_id)
        content_embedding = self.content_embeddings(content_id)

        return self.ratings(tf.concat([user_embedding, content_embedding], axis=1))

In [48]:
# user_id, content_id
RankingDefinitionModel()(([USER_ID], [CONTENT_ID]))

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.00753504]], dtype=float32)>

In [49]:
TASK_RANKING = tfrs.tasks.Ranking(
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=[tf.keras.metrics.RootMeanSquaredError()],
)

In [50]:
class RankingsModel(tfrs.models.Model):
    def __init__(self):
        super().__init__()
        self.ranking_model: tf.keras.Model = RankingDefinitionModel()
        self.task: tf.keras.layers.Layer = TASK_RANKING

    def call(self, features: Dict[str, tf.Tensor]) -> tf.Tensor:
        return self.ranking_model((features["user_id"], features["content_id"]))

    def compute_loss(
        self, features: Dict[Text, tf.Tensor], training=False
    ) -> tf.Tensor:
        labels = features.pop("engaged_pct")

        rating_predictions = self(features)

        # The task computes the loss and the metrics.
        return self.task(labels=labels, predictions=rating_predictions)

### Fit and Evaluate

In [51]:
model_ranking = RankingsModel()
model_ranking.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [52]:
cached_train_ranking = train_ranking.shuffle(100_000).batch(8192).cache()
cached_test_ranking = test_ranking.batch(4096).cache()

In [53]:
model_ranking.fit(cached_train_ranking, epochs=5)

Epoch 1/5
19/19 [==============================] - 2s 58ms/step - root_mean_squared_error: 0.3637 - loss: 0.1309 - regularization_loss: 0.0000e+00 - total_loss: 0.1309
Epoch 2/5
19/19 [==============================] - 0s 25ms/step - root_mean_squared_error: 0.3494 - loss: 0.1217 - regularization_loss: 0.0000e+00 - total_loss: 0.1217
Epoch 3/5
19/19 [==============================] - 0s 25ms/step - root_mean_squared_error: 0.3488 - loss: 0.1213 - regularization_loss: 0.0000e+00 - total_loss: 0.1213
Epoch 4/5
19/19 [==============================] - 0s 18ms/step - root_mean_squared_error: 0.3482 - loss: 0.1208 - regularization_loss: 0.0000e+00 - total_loss: 0.1208
Epoch 5/5
19/19 [==============================] - 0s 20ms/step - root_mean_squared_error: 0.3474 - loss: 0.1202 - regularization_loss: 0.0000e+00 - total_loss: 0.1202


In [54]:
model_ranking.evaluate(cached_test_ranking, return_dict=True)

13/13 [==============================] - 1s 24ms/step - root_mean_squared_error: 0.3481 - loss: 0.1196 - regularization_loss: 0.0000e+00 - total_loss: 0.1196


{'root_mean_squared_error': 0.34809935092926025,
 'loss': 0.10958518087863922,
 'regularization_loss': 0,
 'total_loss': 0.10958518087863922}

### Get Content Recommendations - Ranking

In [57]:
final_ratings = {}

content_ids_from_retrieval = content_ids_recommended[0]
print("content from retrieval (unsorted content_ids): ", content_ids_from_retrieval)

for content_id in content_ids_from_retrieval:
    content_id = content_id.numpy()
    final_ratings[content_id] = model_ranking(
        # user | content_id
        {"user_id": np.array([f"{USER_ID}"]), "content_id": np.array([content_id])}
    )

print("----------------")
print("Ranked Content")
print(f"Ratings for user {USER_ID}:")
print("content_id | score")


for content_id, score in sorted(
    final_ratings.items(), key=lambda x: x[1], reverse=True
):
    print(f"{content_id}: {score}")

content from retrieval (unsorted content_ids):  tf.Tensor(
[b'49a9f915b5' b'a533f2f57a' b'20b0745a31' b'487d868f96' b'19e9799800'
 b'a1671b9953' b'9f943715a3' b'25ad9f4d85' b'd7dad09669' b'73e16a0447'], shape=(10,), dtype=string)
----------------
Ranked Content
Ratings for user 82216a2c7d:
content_id | score
b'73e16a0447': [[0.36834288]]
b'a1671b9953': [[0.367317]]
b'487d868f96': [[0.36708573]]
b'd7dad09669': [[0.36350384]]
b'25ad9f4d85': [[0.35258558]]
b'a533f2f57a': [[0.3521912]]
b'49a9f915b5': [[0.3467191]]
b'19e9799800': [[0.33223924]]
b'20b0745a31': [[0.3312234]]
b'9f943715a3': [[0.32872432]]
